In [106]:
from pathlib import Path
from string import punctuation

import pandas as pd
import psycopg2 as pg

import database

In [2]:
def agg_factory_SELECT(agg):
    return "{feature}.{agg} AS {feature}_{agg}".format(agg=agg, feature="{feature}")

In [ ]:
def find_dupes_QUERY(feature):
    return """
    SELECT
      song,
      segment,
      COUNT(*)
    FROM {feature}
    GROUP BY
      song,
      segment
    HAVING COUNT(*) > 1;
    """

In [4]:
pitches = [f'pitch_{i}' for i in range(12)]
# spectrals = "spectral_bandwidth spectral_centroid spectral_flatness spectral_rolloff".split()
# tonnetzes = "x_perf5 x_min3 x_maj3 y_perf5 y_min3 y_maj3".split()
# features = "tempogram dynamic_tempo rmse zero_crossing_rate".split()
features = "dynamic_tempo".split()
# agg_features = features + spectrals + tonnetzes + pitches
agg_features = features + pitches

In [5]:
def features_aggs_SELECT(features, *aggs):
    aggs = ',\n'.join(agg_factory_SELECT(agg) for agg in aggs)
    return ',\n\n'.join(aggs.format(feature=feature) for feature in features)

In [6]:
def table_others_LEFT_JOIN(table, *others):
    leftjoin = "LEFT JOIN {other} ON {table}.song={other}.song AND " \
               "{table}.segment={other}.segment".format(table=table, other="{other}")  # use () to slit lines?
    
    leftjoins = "\n".join(leftjoin.format(other=other) for other in others)
    return f'FROM {table}\n{leftjoins}'

In [9]:
# NOTE: treat 'tempo' feature differently since it only has one feature -- 'val'

features_SELECT = features_aggs_SELECT(agg_features, 'mean', 'median', 'std') #'mode'
features_LEFT_JOIN = table_others_LEFT_JOIN("tempo", *agg_features)
features_SELECT_LEFT_JOIN = f"SELECT\n{features_SELECT}\n\n{features_LEFT_JOIN}"

features_QUERY = f"""
SELECT
tempo.val AS tempo,
tempo.song AS song,
tempo.segment AS seg,

{features_SELECT}

{features_LEFT_JOIN};
"""

print(features_QUERY)


SELECT
tempo.val AS tempo,
tempo.song AS song,
tempo.segment AS seg,

dynamic_tempo.mean AS dynamic_tempo_mean,
dynamic_tempo.median AS dynamic_tempo_median,
dynamic_tempo.std AS dynamic_tempo_std,

pitch_0.mean AS pitch_0_mean,
pitch_0.median AS pitch_0_median,
pitch_0.std AS pitch_0_std,

pitch_1.mean AS pitch_1_mean,
pitch_1.median AS pitch_1_median,
pitch_1.std AS pitch_1_std,

pitch_2.mean AS pitch_2_mean,
pitch_2.median AS pitch_2_median,
pitch_2.std AS pitch_2_std,

pitch_3.mean AS pitch_3_mean,
pitch_3.median AS pitch_3_median,
pitch_3.std AS pitch_3_std,

pitch_4.mean AS pitch_4_mean,
pitch_4.median AS pitch_4_median,
pitch_4.std AS pitch_4_std,

pitch_5.mean AS pitch_5_mean,
pitch_5.median AS pitch_5_median,
pitch_5.std AS pitch_5_std,

pitch_6.mean AS pitch_6_mean,
pitch_6.median AS pitch_6_median,
pitch_6.std AS pitch_6_std,

pitch_7.mean AS pitch_7_mean,
pitch_7.median AS pitch_7_median,
pitch_7.std AS pitch_7_std,

pitch_8.mean AS pitch_8_mean,
pitch_8.median AS pitch_8_

In [57]:
connection = database.connect_db()

In [12]:
from datetime import datetime

#takes 50 minutes since it is joining things weirdly from 2100 entries to 9 million entries...
print(datetime.now().strftime("%A, %d. %B %Y %I:%M%p"), '\n')
# df = pd.read_sql(features_QUERY, connection)   # uncomment to run at own peril
print(datetime.now().strftime("%A, %d. %B %Y %I:%M%p"), '\n')

Tuesday, 12. February 2019 08:51AM 

Tuesday, 12. February 2019 09:41AM 



In [13]:
# df.to_pickle("joined_df.pkl")

In [15]:
df.shape  # wtf?

(9895717, 42)

In [43]:
df_no_dupes = df.drop_duplicates(subset=['song', 'seg'])

In [44]:
df_no_dupes.shape  # now correct

(2121, 42)

In [45]:
df_no_dupes.to_pickle("joined_df.pkl")

In [46]:
del df

In [99]:
connection.close()

In [95]:
df_no_dupes.head()

,tempo,song,seg,dynamic_tempo_mean,dynamic_tempo_median,dynamic_tempo_std,pitch_0_mean,pitch_0_median,pitch_0_std,pitch_1_mean,...,pitch_8_std,pitch_9_mean,pitch_9_median,pitch_9_std,pitch_10_mean,pitch_10_median,pitch_10_std,pitch_11_mean,pitch_11_median,pitch_11_std
0,161.499023,Infinite,0,131.606177,117.453835,21.591310,0.478847,0.436120,0.378404,0.396772,...,0.226710,0.158108,0.102223,0.137175,0.323886,0.108240,0.345245,0.237460,0.176930,0.171720
1,161.499023,Infinite,1,156.357925,161.499023,19.373141,0.535089,0.550514,0.360815,0.460853,...,0.216130,0.219131,0.150535,0.189497,0.368779,0.223701,0.338631,0.272720,0.188502,0.176208
2,161.499023,Infinite,2,161.499023,161.499023,0.000000,0.532976,0.535744,0.353502,0.458424,...,0.190942,0.233574,0.166762,0.190307,0.395313,0.242320,0.332752,0.293611,0.232436,0.176653
3,161.499023,Infinite,3,161.499023,161.499023,0.000000,0.560920,0.588350,0.356478,0.466850,...,0.238066,0.227487,0.155340,0.200086,0.386434,0.262794,0.347692,0.293144,0.206469,0.197045
4,161.499023,Infinite,4,151.700483,161.499023,24.860557,0.553595,0.556621,0.359414,0.455063,...,0.194140,0.274195,0.234160,0.215002,0.440270,0.356976,0.337480,0.306196,0.239138,0.183412


In [82]:
query = """
SELECT
tempo.val AS tempo,
tempo.song AS song,
tempo.segment AS seg,

pitch_0.mean AS pitch_0_mean,
pitch_0.median AS pitch_0_median,
pitch_0.std AS pitch_0_std

FROM tempo
LEFT JOIN pitch_0 ON tempo.song=pitch_0.song AND tempo.segment=pitch_0.segment;
"""

In [88]:
query = """
SELECT
tempo.val AS tempo,
tempo.song AS song,
tempo.segment AS seg

FROM tempo;
"""

In [71]:
query = """
SELECT
pitch_0.song as song,
pitch_0.segment as seg,
pitch_0.mean AS pitch_0_mean,
pitch_0.median AS pitch_0_median,
pitch_0.std AS pitch_0_std

FROM pitch_0;
"""

In [89]:
df_test = pd.read_sql(query, connection)

In [90]:
df_test.shape

(2145, 3)

In [74]:
df_test[df_test.duplicated(subset=['song', 'seg'])]  
# odd that some aggregations of the same feature got different results. 
# Probably happened when I stopped executions

,song,seg,pitch_0_mean,pitch_0_median,pitch_0_std
242,RememberMe,0,0.298921,0.298856,0.137504
243,RememberMe,1,0.285447,0.292834,0.126076
244,RememberMe,2,0.293660,0.247067,0.140996
269,KenKaniff,0,0.343874,0.258309,0.264737
270,KenKaniff,1,0.535997,0.547917,0.255880
330,TheWayIAm,0,0.319524,0.250079,0.217544
331,TheWayIAm,1,0.369592,0.296501,0.215209
332,TheWayIAm,2,0.449232,0.325292,0.288762
333,TheWayIAm,3,0.416388,0.376356,0.247847
334,TheWayIAm,4,0.405014,0.261083,0.280264


In [91]:
df_test.drop_duplicates(subset=['song', 'seg']).shape

(2121, 3)

In [92]:
df_test.drop_duplicates(subset=['song', 'seg'], keep=False).shape

(2099, 3)

In [87]:
2099 / 2121

0.9896275341819897

In [93]:
df_no_dupes.shape

(2121, 42)

In [94]:
2121 - 2099  # only 22 entries that could possibly have inconsitent values

22

In [132]:
cwd = Path.cwd()
tracks_dir = cwd / 'tracks'
data_dir = cwd / 'data'
pkl_path = data_dir / 'audio_df.pkl'

tracks = list(tracks_dir.iterdir())

In [118]:
eminem = 'Eminem'
rhcp = 'RedHotChiliPeppers'

In [122]:
df_no_dupes = df_no_dupes.reset_index(drop=True)

In [126]:
def find_artist(song):
    for track in tracks:
        name = track.name.title().translate(punct_table)
        if song in name:
            if eminem in name:
                return eminem
            elif rhcp in name:
                return rhcp
            else:
                return np.nan

In [128]:
df_no_dupes['artist'] = df_no_dupes.song.map(find_artist)

In [130]:
df_no_dupes.head()

,tempo,song,seg,dynamic_tempo_mean,dynamic_tempo_median,dynamic_tempo_std,pitch_0_mean,pitch_0_median,pitch_0_std,pitch_1_mean,...,pitch_9_mean,pitch_9_median,pitch_9_std,pitch_10_mean,pitch_10_median,pitch_10_std,pitch_11_mean,pitch_11_median,pitch_11_std,artist
0,161.499023,Infinite,0,131.606177,117.453835,21.591310,0.478847,0.436120,0.378404,0.396772,...,0.158108,0.102223,0.137175,0.323886,0.108240,0.345245,0.237460,0.176930,0.171720,Eminem
1,161.499023,Infinite,1,156.357925,161.499023,19.373141,0.535089,0.550514,0.360815,0.460853,...,0.219131,0.150535,0.189497,0.368779,0.223701,0.338631,0.272720,0.188502,0.176208,Eminem
2,161.499023,Infinite,2,161.499023,161.499023,0.000000,0.532976,0.535744,0.353502,0.458424,...,0.233574,0.166762,0.190307,0.395313,0.242320,0.332752,0.293611,0.232436,0.176653,Eminem
3,161.499023,Infinite,3,161.499023,161.499023,0.000000,0.560920,0.588350,0.356478,0.466850,...,0.227487,0.155340,0.200086,0.386434,0.262794,0.347692,0.293144,0.206469,0.197045,Eminem
4,161.499023,Infinite,4,151.700483,161.499023,24.860557,0.553595,0.556621,0.359414,0.455063,...,0.274195,0.234160,0.215002,0.440270,0.356976,0.337480,0.306196,0.239138,0.183412,Eminem


In [133]:
df_no_dupes.to_pickle(pkl_path)